### What is decorator?

It's just syntax sugar!

In [1]:
from functools import lru_cache

@lru_cache
def my_sum(a, b):
    return a + b

print(help(my_sum))

Help on _lru_cache_wrapper in module __main__:

my_sum(a, b)

None


Same as this

In [2]:
from functools import lru_cache

def my_sum(a, b):
    return a + b
my_sum = lru_cache(my_sum)

### Two decorators?

Here's example from flask app

In [3]:
@app.route("/api/ping", methods=["POST"])
@jwt_required
def ping():
    return "pong", 200

NameError: name 'app' is not defined

Is the same as

In [ ]:
def ping():
    return "pong", 200
ping = jwt_required(ping)
ping = app.route("/api/ping", methods=["POST"])(ping)

That's it. Done.
### Implementing own decorator

Well, decorator is something callable, that returns new object that
will be assigned to the same variable name as original function.

Here is fun but useless decorator:

In [4]:
def my_decorator(original_function):
    return "It's just a string now. Ha-ha."


@my_decorator
def sum(a, b):
    return a + b

print(sum)

sum(1, 3)

It's just a string now. Ha-ha.


TypeError: 'str' object is not callable

So to implement some useful decorator you can return some function.

In this case we'll return original functions

In [5]:
registered_functions = []

def func_keeper(original_function):
    registered_functions.append(original_function)
    return original_function

@func_keeper
def sum(a, b):
    return a + b

@func_keeper
def sub(a, b):
    return a - b

print(f'{sum(1, 2)=}')
print(f'{sub(5, 3)=}')

print(f'{registered_functions=}')


sum(1, 2)=3
sub(5, 3)=2
registered_functions=[<function sum at 0x11533dee0>, <function sub at 0x11533df70>]


But now let's modify function behavior, for this we must return new function

In [6]:
def square_all_output(original_function):
    def wrapper(*args, **kwargs):
        result = original_function(*args, **kwargs)
        return result ** 2
    return wrapper # we return new function


@square_all_output
def sum(a, b):
    """Sum two numbers"""
    return a + b

print(f'{sum(1, 2)=}')
print(f'{sum(3, 3)=}')


sum(1, 2)=9
sum(3, 3)=36


In [7]:
# Check the function name!
print(sum.__name__)
print(sum.__doc__)

wrapper
None


It's not our "original_function" it's another one that uses original.

But it's kinda ugly, we would want to new function to have same name and docstring.

We just need to use another decorator :)

In [8]:
from functools import wraps

def square_all_output(original_function):
    @wraps(original_function)  # it will copy name and docstring to wrapper
    def wrapper(*args, **kwargs):
        result = original_function(*args, **kwargs)
        return result ** 2
    return wrapper # we return new function


@square_all_output
def sum(a, b):
    """Sum two numbers"""
    return a + b

print(sum.__name__)
print(sum.__doc__)


sum
Sum two numbers


### OK. More useful example

In [11]:
import time

def my_timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start_time
        print(f'Function {func.__name__} took {elapsed:.3f}s')
        return result
    return 3

@my_timer
def sum(a, b):
    print('123')
    time.sleep(a)
    return a + b

sum = my_timer(sum)

print(sum())
print(type(sum))
print(f'{sum(1, 2)=}')


3


TypeError: 'int' object is not callable

### Parametrized decorator

It's just a function, that returns a decorator.

And decorator is a function, that returns a function.

So we need to implement function, in function, in function.

In [10]:
def power_all_output(power=2):
    def decorator(original_function):
        def wrapper(*args, **kwargs):
            return original_function(*args, **kwargs) ** power
        return wrapper
    return decorator


@power_all_output(3)
def sum_powered_on_three(a, b):
    return a + b

print(f'{sum_powered_on_three(1, 2)=}')


@power_all_output()  # we still need to call `power_all_output` to get decorator
def sum_powered_on_two(a, b):
    return a + b

print(f'{sum_powered_on_two(1, 2)=}')


sum_powered_on_three(1, 2)=27
sum_powered_on_two(1, 2)=9


In [2]:
def power_all_output(power=2):
    def decorator(original_function):
        def wrapper(*args, **kwargs):
            return original_function(*args, **kwargs) ** power
        return wrapper
    return decorator


def sum_powered_on_three(a, b):
    return a + b

decorator = power_all_output(3)
sum_powered_on_three = decorator(sum_powered_on_three)

Decorator is not a function, it's a callable object.

### Implementing decorator as class

In [6]:
class Cacher:
    def __init__(self, original_function):
        self.cache = {}
        self.original_function = original_function

    def __call__(self, *args):  # deleted **kwargs for simplicity
        if args not in self.cache:
            self.cache[args] = self.original_function(*args)

        return self.cache[args]


a =    Cacher(sum(1, 2)) # __init__
a()  # __call__
def sum(a, b):
    print(f'sum is called with {a=} {b=}')
    return a + b


print(f'{sum(1, 2)=}')
print(f'{sum(1, 2)=}')  # will be taken from cache
print(f'{sum(2, 2)=}')

TypeError: 'int' object is not callable

Note: For caching, no need to invent bicycle, just use `@functools.lru_cache`

In the case above constructor of the class is a decorator, and class itself is used later.

### Let's create Flask-app-like router

In [17]:
class Router:
    def __init__(self):
        self.routes = {}

    def register(self, path):
        def decorator(original_function):
            self.routes[path] = original_function
            return original_function
        return decorator

    def execute(self, path, *args):
        return self.routes[path](*args)

router = Router()

@router.register('/api/sum')
def sum(a, b):
    return a + b

@router.register('/api/sub')
def sub(a, b):
    return a - b

print(router.routes)

print(f"{router.execute('/api/sum', 2, 3)=}")
print(f"{router.execute('/api/sub', 3, 2)=}")


{'/api/sum': <function sum at 0x115363ca0>, '/api/sub': <function sub at 0x115363b80>}
router.execute('/api/sum', 2, 3)=5
router.execute('/api/sub', 3, 2)=1


You can apply decorator on the class, methods, function.

Here is some examples

In [18]:
from dataclasses import dataclass
from functools import lru_cache

@dataclass
class MyClass:

    # you can't apply decorator here
    attribute1 = 23

    @lru_cache
    def sum(self, a, b):
        return a + b

    @staticmethod
    def sub(a, b):
        return a - b

    @lru_cache
    def __call__(self, a, b):
        return a ** b

    @property
    def attr1(self):
        return self.attribute1


## Tricky question

> Question: Why does it print the execution time even though the decorator's __call__ method is not explicitly invoked?

In [3]:
class MeasureTime:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        import time
        start_time = time.time()
        result = self.func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution time: {execution_time} seconds")
        return result

@MeasureTime
def calculate_sum(n):
    return sum(range(n))

result = calculate_sum(1000000)
print(result)

# Parameterized class in decorator
# Написать декоратор, который будет считать количество вызовов функций этого класса у всех инстансев
class Param:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        result = self.func(*args, **kwargs)
        return result

Execution time: 0.047132253646850586 seconds
499999500000
